# Carregando bibliotecas!**

In [ ]:
# Primeiro tem que carregar todas as bibliotecas. Em python é assim, fazer o quê né
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

Hora de ler os dados

In [ ]:
data = pd.read_csv('../input/housing.csv')
data.info()

Olha os dados aí!

In [ ]:
print ("Jarvis, descreva os dados para mim : \n\n", data.describe())

In [ ]:
data.head()

Vamos só dar uma olhada para ver se está tudo bem

In [ ]:
data.hist(figsize=(20,15), color = 'green')
plt.show()

> # Verificar NULOS

In [ ]:
print('Jarvis? Temos valores nulos no dataset? \n')
print(data.isnull().sum())     

Vamos excluir 

In [ ]:
# Deletando valores NULOS

In [ ]:
data = data.dropna(axis=0)
print("Jarvis, valores Nulos deletados, senhor : ", data.shape)

In [ ]:
print('Jarvis, sobraram valores nulos??!?!?! \n')
print(data.isnull().sum())  

# Entrada e Saída

In [ ]:
# Removendo as colunas ['median_house_value', ocean_proximity]
x_data = data.drop(data.columns[[8, 9]], axis = 1)

y_data = data['median_house_value']

In [ ]:
x_data.head()

In [ ]:
y_data.head()

# Train Test Split (Divisão do resultado em treino e teste)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.30, random_state=101)

# Escalando os dados

Ao passo que não é necessário, facilita tudo... Pois fixa o valor de memória necessário, e não há perda significativa de informação se você usar as casas decimais devidas.

In [ ]:
scaler = MinMaxScaler()
scaler.fit(x_train)

In [ ]:
x_train = pd.DataFrame(data = scaler.transform(x_train), columns = x_train.columns, index= x_train.index)

In [ ]:
x_train.head()

In [ ]:
x_test = pd.DataFrame(data = scaler.transform(x_test), columns = x_test.columns, index= x_test.index)

In [ ]:
x_test.head()

# Convertendo as colunas para o tensorflow

In [ ]:
data.columns

In [ ]:
longitude = tf.feature_column.numeric_column('longitude')
latitutde = tf.feature_column.numeric_column('latitude')
age = tf.feature_column.numeric_column('housing_median_age')
rooms = tf.feature_column.numeric_column('total_rooms')
bedroom = tf.feature_column.numeric_column('total_bedrooms')
pop = tf.feature_column.numeric_column('population')
households = tf.feature_column.numeric_column('households')
income = tf.feature_column.numeric_column('median_income')

In [ ]:
# Aggregating the feature columns
feat_cols = [longitude, latitutde, age, rooms, bedroom, pop, households, income]

In [ ]:
feat_cols

# Criando a entrada

In [ ]:
input_func = tf.estimator.inputs.pandas_input_fn(x = x_train, 
                                                 y = y_train, 
                                                 batch_size = 20, 
                                                 num_epochs = 2000, 
                                                 shuffle = True)

# Cria o modelo usando Deep Neural Nets Regression 

In [ ]:
model = tf.estimator.DNNRegressor(hidden_units = [8, 8, 8, 8, 8], 
                                  feature_columns = feat_cols)

# Treinando o modelo usando 50000 passos

Vamos usar aquela divisão de TREINO, e mandar o modelo achar a melhor regressão

In [ ]:
model.train(input_fn = input_func, steps = 50000)

# Prevendo o resultado

Ou seja, usando aquela divisão de teste!

In [ ]:
predict_input_func = tf.estimator.inputs.pandas_input_fn(x = x_test, 
                                                         batch_size = 20, 
                                                         num_epochs = 1, 
                                                         shuffle = False)

In [ ]:
pred_gen = model.predict(predict_input_func)    

In [ ]:
predictions = list(pred_gen) 

In [ ]:
predictions

In [ ]:
final_y_preds = []

for pred in predictions:
    final_y_preds.append(pred['predictions'])

In [ ]:
final_y_preds

In [ ]:
#  RMSE final Usando o Regressor DNN

In [ ]:
mean_squared_error(y_test, final_y_preds) ** 0.5

                         RSME (Raiz quadrada do erro-médio)  = 72474.108532716637
                -> Rede neural com 4 camadas, Cada uma tendo 8 entradas (DIMENSÕES)